In [1]:
import pandas as pd
import numpy as np
import shap
import gc
from plotly.subplots import make_subplots
from sklearn.model_selection import GridSearchCV
import plotly.graph_objects as go
import pickle

pd.set_option('display.max_rows', 160)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 30)

import warnings
warnings.filterwarnings("ignore")

import random

# from tensorflow.python.keras.models import Sequential, model_from_json
# from tensorflow.python.keras.layers import Dense, LSTM, Dropout
# from tensorflow.python.keras import optimizers
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import optimizers
#from tensorflow.python.keras.optimizers import adam

In [2]:
import tensorflow as tf

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
#data = pd.read_pickle('df_completeMy.pkl')
with open('df_completeMy.pkl', "rb") as fh:
    data = pickle.load(fh)
#del data['subtype']

X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
#free memory
del data
gc.collect();

In [5]:
X_train.shape

(6115148, 74)

In [6]:
Y_train.shape

(6115148,)

In [7]:
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))

In [8]:
print("Train set reshaped", X_train.shape)
print("Validation set reshaped", X_valid.shape)

Train set reshaped (6115148, 74, 1)
Validation set reshaped (238172, 74, 1)


In [10]:
### Create LSTM
serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1

epochs = 20
batch = 128
lr = 0.0001
# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

adam = optimizers.Adam(lr)
my_model.compile(loss='mse', optimizer=adam)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
# lstm_history = lstm_model.fit(X_train, Y_train, 
#                               validation_data=(X_valid, Y_valid), 
#                               batch_size=batch, 
#                               epochs=epochs, 
#                               verbose=2)
my_model.fit(X_train,Y_train,batch_size = 4096,epochs = 10)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/10
1493/1493 [==============================] - 1422s 952ms/step - loss: 1.2065
Epoch 2/10
1493/1493 [==============================] - 1329s 890ms/step - loss: 0.9727
Epoch 3/10
1493/1493 [==============================] - 1431s 959ms/step - loss: 0.9409
Epoch 4/10
1493/1493 [==============================] - 1490s 998ms/step - loss: 0.9241
Epoch 5/10
1493/1493 [==============================] - 31653s 21s/step - loss: 0.9105
Epoch 6/10
1493/1493 [==============================] - 1612s 1s/step - loss: 0.8964
Epoch 7/10
1493/1493 [==============================] - 1499s 1s/step - loss: 0.8860
Epoch 8/10
1493/1493 [==============================] - 2143s 1s/step - loss: 0.8793
Epoch 9/10
1493/1493 [==============================] - 1578s 1s/step - loss: 0.8730
Epoch 10/10
1493/1493 [==============================] - 3341s 2s/step - loss: 0.86

In [13]:
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test_reshaped.shape)

(214200, 74, 1)


In [14]:

submission = pd.read_csv('sample_submission.csv')
submission['item_cnt_month'] =my_model.predict(X_test_reshaped).clip(0,20)
submission[['ID', 'item_cnt_month']].to_csv('LSTM2_submission.csv', index=False)